In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# 加载预训练模型
model_name = "shared-nvme/llm_models/Qwen2.5-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(model_name)

# 加载 LoRA 模型
lora_model_path = "shared-nvme/finetuned_llm_models/sft_lora_qwen_beautiful/"
model = PeftModel.from_pretrained(base_model, lora_model_path)

# 合并 LoRA 参数
model = model.merge_and_unload()

# 保存合并后的模型
merged_model_path = "merged_model"
model.save_pretrained(merged_model_path)
tokenizer.save_pretrained(merged_model_path)

/home/pod/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:27<00:00,  7.00s/it]


('merged_model/tokenizer_config.json',
 'merged_model/special_tokens_map.json',
 'merged_model/vocab.json',
 'merged_model/merges.txt',
 'merged_model/added_tokens.json',
 'merged_model/tokenizer.json')

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 10000 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "shared-nvme/finetuned_llm_models/sft_lora_qwen_beautiful/",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/pod/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Unsloth: Your Flash Attention 2 installation seems to be broken?
A possible explanation is you have a new CUDA version which isn't
yet compatible with FA2? Please file a ticket to Unsloth or FA2.
We shall now use Xformers instead, which does not have any performance hits!
We found this negligible impact by benchmarking on 1x A100.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.11.9: Fast Qwen2 patching. Transformers = 4.46.3.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.684 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 8.6. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.63s/it]
Unsloth 2024.11.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 3584, padding_idx=151665)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3584, out_features=3584, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3584, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora

In [2]:
model.save_pretrained_merged("merged_model", tokenizer, save_method = "merged_16bit",)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 360.78 out of 503.72 RAM for saving.


100%|██████████| 28/28 [00:01<00:00, 24.44it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...


SafetensorError: Error while serializing: IoError(Os { code: 28, kind: StorageFull, message: "No space left on device" })

In [1]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer
base_model_path = "shared-nvme/llm_models/Qwen2.5-7B-Instruct"
lora_model_path = "shared-nvme/finetuned_llm_models/lora_Qwen2.5-7B_beautiful"  # LoRA 微调模型路径

# 加载配置
max_seq_length = 10000  # 可支持自动 RoPE Scaling
dtype = None  # None 表示自动检测数据类型
load_in_4bit = True  # 是否使用 4bit 量化加载模型

# 加载模型
model = AutoPeftModelForCausalLM.from_pretrained(
    lora_model_path,  # LoRA 微调模型路径
    load_in_4bit=load_in_4bit,
)
model = model.merge_and_unload()  # 合并 LoRA 参数到基础模型
model.eval()

# 加载分词器
tokenizer = AutoTokenizer.from_pretrained(base_model_path)  # 分词器应与基础模型匹配

/home/pod/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 4/4 [00:31<00:00,  7.78s/it]
/home/pod/.local/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:336: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [4]:
messages = [                    # Change below!
    {"role": "user", "content": "你是谁"},
#     {"role": "user", "content": "Continue the fibonacci sequence! Your input is 1, 1, 2, 3, 5, 8,"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
answer = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

generated_text = tokenizer.decode(answer[0], skip_special_tokens=True)
print(generated_text)

我是个脏话学习助手，专门教你各种脏话的 usage。有什么问题尽管问，别客气！<|endoftext|>
Below are some instructions that describe some tasks. Write responses that appropriately complete each request.

### Instruction:
你是谁

### Response:
我是个脏话学习助手，专门教你各种脏话的 usage。有什么问题尽管问，别客气！
